## Main Idea

Here we impliment the following idea: Given a 2D Hexagonal lattice, consider the (infinite) set of moves where every lattice point is involved in a pairwise (CCW or CW) exchange with one of its neighbors. We would like to see what combination of such moves, once repeated indefinately, results in the largest exponential rate of increase in the length of generic material lines.  Since there are an infinite number of possible ways to spatially arrange the switches in each move (due to the infinite lattice, infinite geometric configurations, and that each switch could be CW or CCW), we will try to answer a more constrained problem.  We can enumerate the number of unique moves for periodic moves of a given periodic domain (we consider a lattice on a torus).  

<img src="Pictures/HexLattice.jpg">

Here we consider the square lattice compatible with the torus that has two points (see below).  We use wrap-around conditions that make this a torus (here we are using the hexagon with identified opposite sides as the fundamental domain of the tours).  

<img src="Pictures/HexTorusLattice.jpg">

We will label the 3 edges (1-3 for the lattice edges) as follows (along with three more edges 4-6 to get a triangulation):

<img src="Pictures/LabeledTriangulation.jpg">

### Generators

We will refer to individual pair switches by the edge connecting the pair of points (there are three), and a $\pm$ exponent to denote a counter clockwise (CCW $+$) or clockwise (CW $-$) exchange.  So, the 6 generators are: $\sigma^{\pm}_{i}$ for $i \in [1,3]$

Here we can only execute one generator at a time, so the usual notion of an operator is equivalent to that of a genereator.

We will try to find the Braid word up to length 8 on these 6 operators which maximizes the topological entropy per operator.  We will sucessively check the low length braid words first.  We will try larger length braids on a smaller subset of possibilities. 

<img src="Pictures/LabeledTriangulation.jpg">

## Symmetries

Using the above image, we can write down the coordinate transformations corresponding to symmetries.  Here we consider the symmetries to act on the underlying triangulation, and not the torus (as given by the Hexagonal fundamental domain).  The symmetries are: $R$ for a CCW rotation by $2\pi/3$ about either point (give the same transformation), $\overline{R}$ for a CW rotation by $2\pi/3$, and a mirror inversion about the vertical center line $M$.  For $i \in [1,2,3,4,5,6]$, we give the action of the symmetry by the permutation $\pi(i)$. 



$R: \pi = (3,1,2,6,4,5)$  This has $R^3 = \mathbb{1}$ and $R^2 = \overline{R}$

$\overline{R}: \pi = (2,3,1,5,6,4)$

$M: \pi = (1,3,2,5,4,6)$  Note that this will switch CCW and CW braid generators.  Also $M^2 = \mathbb{1}$

<img src="Pictures/LabeledTriangulation.jpg">

## Triangulation Coordinates

We encode the way curves wind around the lattice points by specifying a triangulation.  In this case we need three more edges to create a triangulation (see above). Each edge weight counts the number of transverse intersections of the curves.  

For our initial curve, we will take the band $E = (0,2,2,2,2,2)$ - the band enclosing edge 1.  If the braid is pA, then this should stretch out exponentially.  The asymptotic weighted traintrack that results will not depend on this initial condition for a pA braid.

For the action of the symmetry operators on the triangulation coordinates, we will only consider $M$ and $R,\overline{R}$.

## Symmetry Operators and Braid Generators

We will fully define the action of one generator on the coordinates ($S = \sigma^{+}_1$).  All other generators will be related to this one by conjugating with a set of symmetries.  The operators act via left action (right most operator acts first).


$\sigma^{+}_1 = S$   

$\sigma^{-}_1 = MSM$   

$\sigma^{+}_2 = RS\overline{R}$

$\sigma^{-}_2 = RMSM\overline{R}$

$\sigma^{+}_3 = \overline{R}SR$

$\sigma^{-}_3 = \overline{R}MSMR$


<img src="Pictures/LabeledTriangulation.jpg">

## Coordinate Update Rules

First for the update rule for $S$, which are constructed by breaking down the point interchange into a series of Whitehead moves and using our edge updating formula at each step.  Here is the formula for a Whitehead move (where E is the edge between the two triangles, A,B,C,D are the edges of the quadrilateral in cyclic order, and E' is the new edge after the flip):

$E' = \max(A+C,B+D) - E \equiv \Delta(A,B,C,D;E)$

Now we use this to construct the overall update rule for $S$.  Consider the following figure. 

<img src="Pictures/FlipSequence2.jpg">


#### Rules for $S$

$(E^n = S E)$  :  $E^n_i = E^{*}_{\pi(i)}$, where $E^{*} = (E_1,E''_2,E''_3,E''_4,E'_5,E'_6)$ and $\pi = (1,6,5,3,4,2)$ (i.e. no permutation needed)

$E'_2 = \Delta(E_1,E_5,E_6,E_3;E_2)$

$E'_4 = \Delta(E_1,E_3,E_5,E_6;E_4)$

$E'_3 = \Delta(E_1,E'_2,E_5,E'_4;E_3)$

$E'_6 = \Delta(E_1,E'_4,E'_2,E_5;E_6)$

$E'_5 = \Delta(E_1,E'_6,E'_3,E'_2;E_5)$

$E''_2 = \Delta(E_1,E'_5,E'_6,E'_4;E'_2)$

$E''_3 = \Delta(E'_4,E_1,E'_5,E'_6;E'_3)$

$E''_4 = \Delta(E_1,E''_2,E''_3,E'_6;E'_4)$

## Function Definitions

In [1]:
#Now we set up the functions, S, M, R, R-inv.  Each of them take in a numpy array of length 6 and output the same data type.
import numpy as np
import copy

#this is the fundamental function that updates the central edge in the Whitehead move
def Delta(A,B,C,D,E):
    return max(A+C,B+D) - E

#this updates the state of the triangulation vector for the CCW switch connectinging A and B along edge 2 (and outputs the new one).  Notice that the indexing is one less than in the notes (starts with 0)
def S_switch(WS):

    E1p = Delta(WS[0],WS[4],WS[5],WS[2],WS[1])

    E3p = Delta(WS[0],WS[2],WS[4],WS[5],WS[3])
    
    E2p = Delta(WS[0],E1p,WS[4],E3p,WS[2])

    E5p = Delta(WS[0],E3p,E1p,WS[4],WS[5])

    E4p = Delta(WS[0],E5p,E2p,E1p,WS[4])

    E1pp = Delta(WS[0],E4p,E5p,E3p,E1p)

    E2pp = Delta(E3p,WS[0],E4p,E5p,E2p)
    
    E3pp = Delta(WS[0],E1pp,E2pp,E5p,E3p)

    return np.array([WS[0],E5p,E4p,E2pp,E3pp,E1pp])


#now for the Mirror flip about the horizontal axis
def M_flip(WS):
    return np.array([WS[0],WS[2],WS[1],WS[4],WS[3],WS[5]])

#CCW rotation
def R_rot(WS):
    return np.array([WS[2],WS[0],WS[1],WS[5],WS[3],WS[4]])

#CW rotation
def RInv_rot(WS):
    return np.array([WS[1],WS[2],WS[0],WS[4],WS[5],WS[3]])

                     
#now for the individual generators
def Sig1(WS,Positive = True):
    if Positive:
        return S_switch(WS)
    else:
        return M_flip(S_switch(M_flip(WS)))
                    
def Sig2(WS,Positive = True):
    if Positive:
        return R_rot(S_switch(RInv_rot(WS)))
    else:
        return R_rot(M_flip(S_switch(M_flip(RInv_rot(WS)))))     
                                          
def Sig3(WS,Positive = True):
    if Positive:
        return RInv_rot(S_switch(R_rot(WS)))
    else:
        return RInv_rot(M_flip(S_switch(M_flip(R_rot(WS)))))                                   
                     
                     
def Generator(WS,n,Positive = True):
    switcher = {
        1: lambda WSin,Pos:Sig1(WSin,Pos),
        2: lambda WSin,Pos:Sig2(WSin,Pos),
        3: lambda WSin,Pos:Sig3(WSin,Pos)
    }
    return switcher.get(n)(WS,Positive)


Now we have all the functions defined that we need, and we can move on to lattice braid words acting on the triangluation coordinates

In [2]:
#First, we will wrap out definition of a lattice braid generator as a list [i,j], where j are True or False (CCW or CW), and i are the move subscripts (1-4)
def Lattice_Braid_Operator(WS, GenInfo):
    WSout = Generator(WS,GenInfo[0],GenInfo[1])
    return WSout
    

#now a braid word is a list of the Generator info elements.  This function takes in such a list and outputs the triangulation coordinates after applying each of the generators (in index order: 0, 1, 2, ...)
def Lattice_Braid_Action(WS,LatticeBraid):
    WSout = copy.copy(WS)
    for i in range(len(LatticeBraid)):
        WSout = Lattice_Braid_Operator(WSout,LatticeBraid[i])
    return WSout

#We also need a function that gets the total weight of the triangulation coordinates (just sum of all weights)
def Weight_Total(WS):
    wtot = 0
    for i in range(len(WS)):
        wtot += WS[i]
    return wtot

#now let's generate a list of the 6 generators
G = [[1,True],[1,False],[2,True],[2,False],[3,True],[3,False]]

<img src="Pictures/LabeledTriangulation.jpg">

In [3]:
#Let's try this out.  These are bands about each pair connected by an edge
WSvals = [np.array([0,2,2,2,2,2]),np.array([2,0,2,2,2,2]),np.array([2,2,0,2,2,2])]

GenPos = [[i+1,True] for i in range(3)]
GenNeg = [[i+1,False] for i in range(3)]

#let's cycle through all the generators and have them act on the bands that should be invariant as a check

for i in range(3):
    print(i, WSvals[i],Lattice_Braid_Operator(WSvals[i],GenPos[i]),Lattice_Braid_Operator(WSvals[i],GenNeg[i]))

print("\n")

for i in range(3):
    print(i, WSvals[i],Lattice_Braid_Operator(WSvals[i],GenPos[(i+1)%3]),Lattice_Braid_Operator(Lattice_Braid_Operator(WSvals[i],GenPos[(i+1)%3]),GenNeg[(i+1)%3]))
#now let's check them against bands that should actually change, and impliment the inverse

0 [0 2 2 2 2 2] [0 2 2 2 2 2] [0 2 2 2 2 2]
1 [2 0 2 2 2 2] [2 0 2 2 2 2] [2 0 2 2 2 2]
2 [2 2 0 2 2 2] [2 2 0 2 2 2] [2 2 0 2 2 2]


0 [0 2 2 2 2 2] [4 2 2 4 2 4] [0 2 2 2 2 2]
1 [2 0 2 2 2 2] [2 4 2 4 4 2] [2 0 2 2 2 2]
2 [2 2 0 2 2 2] [2 2 4 2 4 4] [2 2 0 2 2 2]


All of these checks work, and the functions appear to be doing what they are designed to do.  Now we would like to find the exponential stretching rate for general braids.

## Getting the Topological Entropy

In [4]:
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import math
from scipy.optimize import curve_fit

#Let's automate this.  Creating a function that will output the braiding entropy and fit
def linear_func(x, a, b):
    return a*x+b

def GetTE(Bin):
    WS = np.array([0.0,2.0,2.0,2.0,2.0,2.0])
    Length = []
    Iterations = []
    Length.append(Weight_Total(WS))
    Iterations.append(0)
    numiter = 100
    for i in range(numiter):
        WS = Lattice_Braid_Action(WS,Bin)
        Length.append(Weight_Total(WS))
        Iterations.append(Iterations[-1]+len(Bin))

    LWeights = [np.log(Length[i]) for i in range(0,len(Length))]
    indend = len(Length)-1
    fracstart = 2
    indst = int(indend/fracstart)
    popt, pcov = curve_fit(linear_func, Iterations[indst:indend], LWeights[indst:indend])  #fitting to a linear function ax+b
    #popt has the optimal fits for a and b (in that order), and pcov has the covariance
    perr = np.sqrt(np.diag(pcov))  #the one standard deviation errors
    return [popt[0], perr[0]]

#This one is better
def GetTE2(Bin, tolerance = 1e-10, numitermax = 100,Verbose = False):
    WS = np.array([0.0,2.0,2.0,2.0,2.0,2.0])
    NumGen = len(Bin)
    numitermin = 6
    for i in range(numitermin):
        WS = Lattice_Braid_Action(WS,Bin)
    LogWeight = np.log(Weight_Total(WS))    
    LogWeightPrev = 0
    
    iternum = numitermin
    TE = (LogWeight - LogWeightPrev)/NumGen
    TEprev = 0
    
    while np.abs(TE - TEprev) > tolerance and iternum < numitermax:
        iternum += 1
        WS = Lattice_Braid_Action(WS,Bin)
        LogWeightPrev = LogWeight
        TEprev = TE
        #print(Weight_Total(WS))
        LogWeight = np.log(Weight_Total(WS))
        TE = (LogWeight - LogWeightPrev)/NumGen

    if Verbose:
        if iternum == numitermax:
            print("Braiding Entropy of ", TE, " with tolerance of ", np.abs(TE - TEprev), " after the maximum of ", iternum, " iterations")
        else:
            print("Braiding Entropy of ", TE, " with tolerance of ", np.abs(TE - TEprev), " after ", iternum, " iterations")
    return [TE, np.abs(TE - TEprev)]

In [5]:
print(GetTE([[1, True], [3, False], [2, True]]))
print(GetTE2([G[0],G[4],G[5]],numitermax = 8,Verbose = True))

[0.8779719312832112, 2.6801605489768083e-17]
Braiding Entropy of  0.0  with tolerance of  0.0  after the maximum of  8  iterations
[0.0, 0.0]


## Checking braid words by brute force
Here we will run through the all the combinatorial possibilities of braid words up to the length that runtime will allow.

In [6]:
G1 = [[1,True],[1,False]]
G2 = [[2,True],[2,False]]
G3 = [[3,True],[3,False]]

G = G1 + G2 + G3

GN1 = G2+G3
GN2 = G1+G3
GN3 = G1+G2
GN = [GN1,GN2,GN3]

In [7]:
#here we make a recursive function that will do the same thing as nested loops, but out to an arbitrary depth.
import time

def GetTEPObraids(depth_end = 8, BraidIn  = [[1,True]], AccumBraids = [[0,None]]):
    if len(BraidIn) < depth_end:
        #add endings to the current braid and pass through this function
        for i in range(len(G)):
            BraidOut = BraidIn + [G[i]]
            AccumBraids = GetTEPObraids(depth_end,BraidOut,AccumBraids)

    else:
        #halting condition
        #find the topological entropy for this braid
        #return the accumulated braid list with the new braid if it has 
        latestMaxTE = AccumBraids[-1][0]
        TEtemp = GetTE2(BraidIn,numitermax = 10)[0]

        if TEtemp >= (latestMaxTE-0.0001):
            if TEtemp <= (latestMaxTE+0.0001):
                AccumBraids.append([TEtemp,BraidIn])
            else:
                AccumBraids = [[TEtemp,BraidIn]]
    return AccumBraids
    

In [12]:
def CounterToStr(countin):
    if countin < 10:
        return "000" + str(countin)
    elif countin < 100:
        return "00" + str(countin)
    elif countin < 1000:
        return "0" + str(countin)
    elif countin < 10000:
        return str(countin)
    else:
        return "countertoobig"

In [ ]:
timelimit = 60*60*10  #10 hours (in seconds)
#timelimit = 60*2
base = "Hex2ptmaxTEPObraidsofLen"
ending = ".txt"

braidlen = 2
timeout = False
while not timeout:

    filename = base + CounterToStr(braidlen) + ending
    fileOut = open(filename,"w")
    fileOut.write("Max TEPO Braids and TEPO value for braids of length "+str(braidlen)+": \n")
    timestart = time.time()
    AB = GetTEPObraids(braidlen)
    timeend = time.time()
    for i in range(len(AB)):
        fileOut.write(str(AB[i][0])+" "+str(AB[i][1])+"\n")
    fileOut.close()

    braidlen += 1
    if abs(timeend-timestart)*len(G) > timelimit:
        timeout  = True


In [13]:
def GetTEPObraids2(depth_end = 8, BraidIn  = [[1,True]], AccumBraids = [[0,None]]):

    if len(BraidIn) < depth_end:
        #add endings to the current braid and pass through this function
        indlast = BraidIn[-1][0]-1
        for i in range(len(GN[indlast])):
            BraidOut = BraidIn + [GN[indlast][i]]
            AccumBraids = GetTEPObraids2(depth_end,BraidOut,AccumBraids)

    else:
        #halting condition
        #find the topological entropy for this braid
        #return the accumulated braid list with the new braid if it has 
        latestMaxTE = AccumBraids[-1][0]
        TEtemp = GetTE2(BraidIn,numitermax = 10)[0]

        if TEtemp >= (latestMaxTE-0.0001):
            if TEtemp <= (latestMaxTE+0.0001):
                AccumBraids.append([TEtemp,BraidIn])
            else:
                AccumBraids = [[TEtemp,BraidIn]]
    return AccumBraids

In [15]:
timelimit = 60*60*10  #10 hours (in seconds)
#timelimit = 60*2
base = "Hex2ptmaxTEPObraidsofLen"
ending = "targeted.txt"


timeout = False

while not timeout:

    filename = base + CounterToStr(braidlen) + ending
    fileOut = open(filename,"w")
    fileOut.write("Max TEPO Braids and TEPO value for braids of length "+str(braidlen)+": \n")
    fileOut.write("Targeted Search \n")
    timestart = time.time()
    AB = GetTEPObraids2(braidlen)
    timeend = time.time()
    for i in range(len(AB)):
        fileOut.write(str(AB[i][0])+" "+str(AB[i][1])+"\n")
    fileOut.close()

    braidlen += 1
    if abs(timeend-timestart)*len(GN[0]) > timelimit:
        timeout  = True
print(braidlen)

[[0.962423650119207,
  [[1, True], [3, True], [2, True], [1, True], [3, True], [2, True]]]]